## Option 2 - Bedrock Knowledge Base Managed RAG with Aurora Vector Store

Prerequisites before you run these scripts : 
1. Deploy an Aurora PostgreSQL Cluster with RDS Data API enabled. You can make use of the CDK stack [here](../cdk/README.md). 
2. Create the vector db schema, table & index using [aws-managed/1_build_vector_db_on_aurora.sql](1_build_vector_db_on_aurora.sql)
3. Note the cluster ARN from the Aurora PostgreSQL Cluster
4. Note the secret Key ARN for the Aurora cluster database username/password.
5. Create a Secrets Manager secret key for the database user bedrock_user (used for RLS)

#### Install the boto3 library.

In [ ]:
%pip install -U boto3==1.34.84

### Restart the Kernel

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

### Imports and clients bedrock_agent, bedrock-agent-runtime, bedrock-runtime,S3

In [ ]:
import boto3
import json
import time
import warnings
warnings.filterwarnings('ignore')

region_name = "us-west-2"

bedrock_agent_client = boto3.client(
    service_name="bedrock-agent", region_name=region_name
)
bedrock_agent_runtime = boto3.client(
    service_name="bedrock-agent-runtime", region_name=region_name
)
bedrock_runtime = boto3.client(service_name="bedrock-runtime", region_name=region_name)

s3_client = boto3.client(service_name="s3", region_name=region_name)

iam = boto3.client('iam')


### Update ARNs & variables from Prerequisites

In [ ]:
# Update the ARN for Aurora Cluster and Secrets for the user bedrock_user
rds_aurora_cluster_arn = "<update aurora cluster arn>"
bedrock_user_secret_arn = "<update bedrock_user secret ARN>"
database_name = "postgres"

account_id = boto3.client("sts").get_caller_identity().get("Account")
bucket_name = f"multi-tenant-home-survey-reports-{account_id}"
embedding_model_id = "amazon.titan-embed-text-v1"

### Function upload document to S3 

In [ ]:
# Function to upload a document to S3
def upload_file_to_s3(file_name, bucket, object_name=None):
    if object_name is None:
        object_name = file_name
    try:
        s3_client.upload_file(file_name, bucket, object_name)
        print(f"File '{file_name}' uploaded to '{bucket}/{object_name}' successfully.")
    except Exception as e:
        print(f"Error uploading file '{file_name}' to '{bucket}/{object_name}': {e}")

### Function to create the Bedrock Knowledge Base

In [ ]:
# Function to create the Bedrock Knowledge Base
def create_knowledge_base(
    name, description, roleArn, embeddingModelArn, rdsConfiguration
):
    create_kb_response = bedrock_agent_client.create_knowledge_base(
        name=name,
        description=description,
        roleArn=roleArn,
        knowledgeBaseConfiguration={
            "type": "VECTOR",
            "vectorKnowledgeBaseConfiguration": {
                "embeddingModelArn": embeddingModelArn
            },
        },
        storageConfiguration={"type": "RDS", "rdsConfiguration": rdsConfiguration},
    )
    return create_kb_response["knowledgeBase"]




### Function to create the datasource in Bedrock Knowledge base

In [ ]:
# Function to create the Datasource in Bedrock Knowledge Base
def create_datasource_in_knowledge_base(
    name, description, knowledgeBaseId, s3Configuration
):
    create_datasource_response = bedrock_agent_client.create_data_source(
        name=name,
        description=description,
        knowledgeBaseId=knowledgeBaseId,
        dataSourceConfiguration={"type": "S3", "s3Configuration": s3Configuration},
    )
    return create_datasource_response["dataSource"]


def wait_for_ingestion(start_job_response):
    job = start_job_response["ingestionJob"]
    while job["status"] != "COMPLETE":
        get_job_response = bedrock_agent_client.get_ingestion_job(
            knowledgeBaseId=kb_id,
            dataSourceId=ds_id,
            ingestionJobId=job["ingestionJobId"],
        )
        job = get_job_response["ingestionJob"]
    print(job)


### Function to Invoke Anthrophic Claude LLM on Bedrock

In [ ]:
# Function to invoke the LLM
def generate_message(bedrock_runtime, model_id, system_prompt, messages, max_tokens):

    body=json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": max_tokens,
            "system": system_prompt,
            "messages": messages
        }
    )

    response = bedrock_runtime.invoke_model(body=body, modelId=model_id)
    response_body = json.loads(response.get('body').read())

    return response_body

def invoke_llm_with_rag(messages):
    model_id = 'anthropic.claude-3-haiku-20240307-v1:0'

    response = generate_message (bedrock_runtime, model_id, "", messages, 300)

    return response

### Function to retrieve vector data chunks

In [ ]:
# Function to retrieve chunks from vector store through KB
def retrieve(query, kbId, numberOfResults=5):
    response = bedrock_agent_runtime.retrieve(
        retrievalQuery={"text": query},
        knowledgeBaseId=kbId,
        retrievalConfiguration={
            "vectorSearchConfiguration": {"numberOfResults": numberOfResults}
        },
    )

    return response


### Function to retrieve vector data chunks with filter enabled

In [ ]:
# Function to retrieve chunks from vector store through KB
def retrieve_with_filters(query, kbId, tenantId, numberOfResults=5):
    tenant_filter = {"equals": {"key": "tenantId", "value": tenantId}}
    response = bedrock_agent_runtime.retrieve(
        retrievalQuery={"text": query},
        knowledgeBaseId=kbId,
        retrievalConfiguration={
            "vectorSearchConfiguration": {
                "numberOfResults": numberOfResults,
                "filter": tenant_filter,
            }
        },
    )

    return response

### Create the IAM role and necessary policies for the Bedrock Knowledge base

In [ ]:
# Inject these variable values into the policy templates and generate policies using sed. 
%env region_name = $region_name
%env account_id = $account_id
%env bucket_name = $bucket_name
%env rds_aurora_cluster_arn = $rds_aurora_cluster_arn
%env bedrock_user_secret_arn = $bedrock_user_secret_arn
%env embedding_model_id = $embedding_model_id

!sed -e "s/\#account_id\#/$account_id/" -e "s/\#bucket_name\#/$bucket_name/" policy-templates/bedrock_data_source_permissions_policy.json > bedrock_data_source_permissions_policy.json
!sed -e "s/\#embedding_model_id\#/$embedding_model_id/" -e "s/\#region_name\#/$region_name/" policy-templates/bedrock_model_permissions_policy.json > bedrock_model_permissions_policy.json
!sed -e "s/\#rds_aurora_cluster_arn\#/$rds_aurora_cluster_arn/" policy-templates/bedrock_aurora_cluster_permissions_policy.json > bedrock_aurora_cluster_permissions_policy.json
!sed -e "s/\#bedrock_user_secret_arn\#/$bedrock_user_secret_arn/" policy-templates/bedrock_secrets_permissions_policy.json > bedrock_secrets_permissions_policy.json
!sed -e "s/\#account_id\#/$account_id/" -e "s/\#region_name\#/$region_name/" policy-templates/bedrock_trust_relationship_policy.json > bedrock_trust_relationship_policy.json


In [ ]:
# Create the role and attach policies

# bedrock-kb-service-role
!aws iam create-role \
    --role-name bedrock_kb_service_role \
    --assume-role-policy-document file://bedrock_trust_relationship_policy.json

# bedrock_model_permissions_policy
!aws iam create-policy \
    --policy-name bedrock_model_permissions_policy \
    --policy-document file://bedrock_model_permissions_policy.json

!aws iam attach-role-policy \
    --role-name bedrock_kb_service_role \
    --policy-arn "arn:aws:iam::$account_id:policy/bedrock_model_permissions_policy"

# bedrock_aurora_cluster_permissions_policy
!aws iam create-policy \
    --policy-name bedrock_aurora_cluster_permissions_policy \
    --policy-document file://bedrock_aurora_cluster_permissions_policy.json

!aws iam attach-role-policy \
    --role-name bedrock_kb_service_role \
    --policy-arn "arn:aws:iam::$account_id:policy/bedrock_aurora_cluster_permissions_policy"

# bedrock_secrets_permission_policy
!aws iam create-policy \
    --policy-name bedrock_secrets_permissions_policy \
    --policy-document file://bedrock_secrets_permissions_policy.json

!aws iam attach-role-policy \
    --role-name bedrock_kb_service_role \
    --policy-arn "arn:aws:iam::$account_id:policy/bedrock_secrets_permissions_policy"

# bedrock_data_source_permissions_policy
!aws iam create-policy \
    --policy-name bedrock_data_source_permissions_policy \
    --policy-document file://bedrock_data_source_permissions_policy.json

!aws iam attach-role-policy \
    --role-name bedrock_kb_service_role \
    --policy-arn "arn:aws:iam::$account_id:policy/bedrock_data_source_permissions_policy"

In [ ]:
# Remove all the generated policy json files.
!rm *.json

### Step 1 : Create the Bedrock Knowledge base
The first step is to create the Bedrock Knowledge base.


NOTE : Before your run this cell, ensure that you have gather the configurations from the Aurora PostgreSQL database and update the variables rds_aurora_cluster_arn and bedrock_user_secret_arn in the previous cells. 

In [ ]:
# Step 1 : Create the Bedrock Knowledge Base

rdsConfiguration = {
    "credentialsSecretArn": str(bedrock_user_secret_arn),
    "databaseName": str(database_name),
    "fieldMapping": {
        "metadataField": "metadata",
        "primaryKeyField": "id",
        "textField": "chunks",
        "vectorField": "embedding",
    },
    "resourceArn": str(rds_aurora_cluster_arn),
    "tableName": "aws_managed.kb",
}
chunkingStrategyConfiguration = {
    "chunkingStrategy": "FIXED_SIZE",
    "fixedSizeChunkingConfiguration": {"maxTokens": 512, "overlapPercentage": 20},
}

embeddingModelArn = (
    f"arn:aws:bedrock:{region_name}::foundation-model/amazon.titan-embed-text-v1"
)
name = f"home-survey-reports-knowledge-base"
description = "Home Survey Reports - multi tenant knowledge base."
roleArn = f"arn:aws:iam::{account_id}:role/bedrock_kb_service_role"

kb = create_knowledge_base(
    name, description, roleArn, embeddingModelArn, rdsConfiguration
)
kb_id = kb["knowledgeBaseId"]

time.sleep(20)
print(f"Knowledge Base created with ID: {kb_id}")

get_kb_response = bedrock_agent_client.get_knowledge_base(knowledgeBaseId=kb_id)

### Step 2 : Create a datasource in the Knowledge base
Next let us create the datasource ( S3 bucket) and add it into the Knowledge base configuration.

In [ ]:
# Step 2 : Create a datasource in the knowledge base
s3_client.create_bucket(Bucket=bucket_name)

s3Configuration = {
    "bucketArn": f"arn:aws:s3:::{bucket_name}",
}
ds = create_datasource_in_knowledge_base(name, description, kb_id, s3Configuration)
print(ds)
ds_id = ds["dataSourceId"]
print(f"Datasource created with ID: {ds_id}")

### Step 3 : Add Tenant1 document into the datasource 
Let us add a document for the Tenant1 into the datasource.

In [ ]:
# Step 3 : Add Tenant1 document into the datasource (S3 bucket)
upload_file_to_s3(
    "../multi_tenant_survey_reports/Home_Survey_Tenant1.pdf", bucket_name, object_name="multi_tenant_survey_reports/Home_Survey_Tenant1.pdf"
)

### Step 4: Ingest the document from the datasource into the vector store.
The newly added document from the datasource needs to be ingested into the Vector Database

In [ ]:
# Step 4 : Ingest data from the datasource into the vector store

start_job_response = bedrock_agent_client.start_ingestion_job(
    knowledgeBaseId=kb_id, dataSourceId=ds_id
)
wait_for_ingestion(start_job_response)

print(f"Datasource ingestion completed")

### Step 5: Retrieve the vector data chunks that are similar to the user question.
Now we can retrieve the vector data chunks from the Tenant1 document based on a user question

In [ ]:
# Step 5 : Retrieve
question = "What is the condition of the roof in my survey report ? "
response = retrieve(question, kb_id)

for i in response['retrievalResults']:
    print(f"source_document={i['location']['s3Location']['uri']}")
    print(f"data_chunk={i['content']['text']}")
    print("------------------------------------")


print(f"Step5 - Retrieval of vector data completed")

### Step 6: Augment the prompt with the data chunks retrieved from the vector store

In [ ]:
# Step 6: Augment the prompt
def get_contexts(retrievalResults):
    contexts = []
    for retrievedResult in retrievalResults: 
        contexts.append(retrievedResult['content']['text'])
    return contexts

contexts = get_contexts(response['retrievalResults'])

prompt = f"""
Human: Use the following pieces of context to provide a concise answer to the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
<context>
{contexts}
</context
Question: {question}
Assistant:
"""

### Step 7 : Generate the response from the LLM
Now we can finally generate the response from the LLM using the augmented prompt as input.

In [ ]:
# Step 7 : Generate the response from the LLM
messages=[{ "role":'user', "content":[{'type':'text','text': prompt.format(contexts, question)}]}]
llm_response = invoke_llm_with_rag(messages)
print(llm_response['content'][0]['text'])
print(f"Step7 - Generated response from LLM")

### Step 8: Add more tenants and their documents

Let us upload documents for a few more tenants : Tenant2, Tenant3, Tenant4, Tenant5

In [ ]:
# Step 8 : Add Tenant2, Tenant3, Tenant4 documents into the datasource (S3 bucket)
upload_file_to_s3("../multi_tenant_survey_reports/Home_Survey_Tenant2.pdf", bucket_name, object_name="multi_tenant_survey_reports/Home_Survey_Tenant2.pdf")
upload_file_to_s3("../multi_tenant_survey_reports/Home_Survey_Tenant3.pdf", bucket_name, object_name="multi_tenant_survey_reports/Home_Survey_Tenant3.pdf")
upload_file_to_s3("../multi_tenant_survey_reports/Home_Survey_Tenant4.pdf", bucket_name, object_name="multi_tenant_survey_reports/Home_Survey_Tenant4.pdf")
upload_file_to_s3("../multi_tenant_survey_reports/Home_Survey_Tenant5.pdf", bucket_name, object_name="multi_tenant_survey_reports/Home_Survey_Tenant5.pdf")

print(f"Step8- Uploaded more tenants documents")

### Step 9: Ingest the new tenant documents into the vector store

In [ ]:
# Step 9 : Ingest new documents from the datasource into the vector database

start_job_response = bedrock_agent_client.start_ingestion_job(
    knowledgeBaseId=kb_id, dataSourceId=ds_id
)
wait_for_ingestion(start_job_response)

print(f"Step9 - Ingestion of new documents completed")

### Step 10 : Retrieve the vector data related to Tenant3.
Now we can attempt to retrieve vector data based on a question from Tenant3. 

In [ ]:
# Step 10 : Retrieve the vector data related to the question of Tenant 1
question = "What is the condition of the roof in my survey report for Tenant3? "
response = retrieve(question, kb_id)

print(f"Step10 - Retrieving vector data for Tenant 1 - complete")

### Step 11: Review the results retrieved. 
The response will include data chunks from multiple tenant data. So the question is how do we enforce tenant isolation so that when we retrieve data from the vector database, we are able to limit it to a specific tenants data.

In [ ]:
# Step 11 : Review the results and validate the response. You will observe that the response includes chunks from other tenants as well.

for i in response['retrievalResults']:
    print(f"source_document={i['location']['s3Location']['uri']}")
    print(f"data_chunk={i['content']['text']}")
    print("------------------------------------")

print(f"Step11 - Review the results and validate the response")

### Step 12: Add metadata tagging to each tenant document. 


In [ ]:
# Step 12 : Add metadata tagging to each tenants document
upload_file_to_s3(
    "../metadata_tags/Home_Survey_Tenant1.pdf.metadata.json",
    bucket_name,
    "multi_tenant_survey_reports/Home_Survey_Tenant1.pdf.metadata.json",
)

upload_file_to_s3(
    "../metadata_tags/Home_Survey_Tenant2.pdf.metadata.json",
    bucket_name,
    "multi_tenant_survey_reports/Home_Survey_Tenant2.pdf.metadata.json",
)

upload_file_to_s3(
    "../metadata_tags/Home_Survey_Tenant3.pdf.metadata.json",
    bucket_name,
    "multi_tenant_survey_reports/Home_Survey_Tenant3.pdf.metadata.json",
)

upload_file_to_s3(
    "../metadata_tags/Home_Survey_Tenant4.pdf.metadata.json",
    bucket_name,
    "multi_tenant_survey_reports/Home_Survey_Tenant4.pdf.metadata.json",
)

upload_file_to_s3(
    "../metadata_tags/Home_Survey_Tenant5.pdf.metadata.json",
    bucket_name,
    "multi_tenant_survey_reports/Home_Survey_Tenant5.pdf.metadata.json",
)

print(f"Step12 - Metadata tags for each document added")

### Step 13 : Ingest the metadata tags into the vector store

In [ ]:
# Step 13 : Ingest tags datasource into the vector database

start_job_response = bedrock_agent_client.start_ingestion_job(
    knowledgeBaseId=kb_id, dataSourceId=ds_id
)
wait_for_ingestion(start_job_response)

print(f"Step13 - Ingestion completed for new metadata documents ")

### Step 14: Retrieve the vector data with tenant filtering enabled.
Amazon Bedrock Knowledge Base supports filtering using metadata tags. In the previous step we tagged each document with the tenant-id. During retrieval we can pass a filter configuration using the desired tenant-id to enforce the tenant specific data chunks are only retrieved from the underlying vector database of the knowledge base.  

Review the response to validate that the data chunks retrieved are from the specific tenants document.

In [ ]:
# Step 14 : Retrieve with filter enabled for tenantid=3
question = "What is the condition of the roof in my survey report  ? "
response = retrieve_with_filters(question, kb_id, 3)

for i in response['retrievalResults']:
    print(i['location']['s3Location']['uri'])
    print(f"data_chunk={i['content']['text']}")
    print(f"tenantid={round(i['metadata']['tenantid'])}")
    print("------------------------------------")
